In [10]:
from __future__ import print_function
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit import Chem
from rdkit import RDLogger
import subprocess
import time
import signal
from subprocess import Popen
import warnings
import shlex
import os

lg = RDLogger.logger()
lg.setLevel(RDLogger.ERROR)  # Only show errors, not warnings.

warnings.filterwarnings("ignore")

Create basic functions:

In [11]:
def replacement_v1(f1, f2, before, after): # name_input, name_output
    
    '''the function performs a topologically 
        identicalsubstitution in molecules'''
    
    cmd_pattern = ['replacement ', ' -i ', '', ' -o ', '', ' -before ' + before + ' -after ' + after + ' -all_atoms']
    cmd_pattern[2] = f1
    cmd_pattern[4] = f2
    cmd = ''.join(cmd_pattern)
    args = shlex.split(cmd)
    p = subprocess.Popen(args)
    result = p.communicate(timeout=100)[0]
    

def write_file(file_name, str_cont):
    
    '''the function writes a file with
        the specified extension'''
    
    ff = open(file_name, "w")
    ff.writelines(str_cont)
    ff.close()

def open_file_if_exists(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as file:
            return file.read()
    else:
        return None

It is also the generaly part of alghoritm. We are create help files and do bioisosterics replacement.

In [12]:
SMILES_cont = []
help_cont = []
file = open("Components-pub.sdf", "r")

with Chem.SDWriter('F_H.sdf') as SDFWr:
    for line in file:
        if line.strip() != '$$$$':
            help_cont.append(line)
        else:
            help_set = set()
            write_file("in.mol", help_cont)
            mol = Chem.MolFromMolFile("in.mol")

            if mol is not None:
                molecule_name = mol.GetProp('_Name')
                replacement_v1("in.mol", "out.sdf", "F", "H")

                if os.path.exists('out.sdf') == True:
                    inf = open('out.sdf', 'rb')
                    fsuppl = Chem.ForwardSDMolSupplier(inf)
                    for mol in fsuppl:
                        
                        if mol != None:
                            smi = Chem.MolToSmiles(mol)
                            help_set.add(smi)

                            SDFWr.write(mol)
                    if len(help_set) != 0:
                        SMILES_cont.append([help_set, molecule_name])
                    inf.close()
                    os.remove("out.sdf")
                else:
                    os.remove("in.mol")
            help_cont = []
            
file.close()
            
#os.remove("in.mol")
#print(SMILES_cont)

[19:03:33] Explicit valence for atom # 15 N, 4, is greater than permitted
[19:03:40] ERROR: Cannot process coordinates on line 6
[19:03:40] ERROR: moving to the beginning of the next molecule
[19:03:40] ERROR: Cannot process coordinates on line 29
[19:03:40] ERROR: moving to the beginning of the next molecule
[19:03:40] ERROR: Cannot process coordinates on line 52
[19:03:40] ERROR: moving to the beginning of the next molecule
[19:03:40] ERROR: Cannot process coordinates on line 75
[19:03:40] ERROR: moving to the beginning of the next molecule
[19:04:13] Explicit valence for atom # 15 C, 5, is greater than permitted
[19:04:14] Explicit valence for atom # 7 O, 3, is greater than permitted
[19:04:17] Explicit valence for atom # 17 N, 4, is greater than permitted
[19:04:17] Explicit valence for atom # 27 O, 3, is greater than permitted
[19:04:51] ERROR: Cannot process coordinates on line 11
[19:04:51] ERROR: moving to the beginning of the next molecule
[19:04:51] ERROR: Cannot process coor

Create txt file with smiles and parent molecules IDs:

In [5]:
file = open("F_H.smiles", "a")
for smi in SMILES_cont:
    name = smi[1]
    for ssmi in smi[0]:
        file.writelines(ssmi + "  " + name + "\n")
file.close()